In [1]:
import pandas as pd
import os
import pandas_read_xml as pdx
os.chdir('D:/duckdb/files/RNA2/')
import polars as pl
import winsound

In [34]:
%%time
periodo = '2018_12'
xmlfile = 'OpenData_Aiuti_'+periodo+'.xml'

df = pdx.read_xml(xmlfile, ['LISTA_AIUTI'], encoding='utf-8') #  latin-1
# df = pdx.flatten(df)
df = pdx.fully_flatten(df)

df.rename(columns = {
'AIUTO|CAR' : 'CAR',
'AIUTO|TITOLO_MISURA' : 'MISURA',
'AIUTO|DES_TIPO_MISURA' : 'DES_TIPO_MISURA',
'AIUTO|COD_CE_MISURA' : 'COD_CE_MISURA',
'AIUTO|BASE_GIURIDICA_NAZIONALE' : 'BASE_GIURIDICA_NAZIONALE',
'AIUTO|IDENTIFICATIVO_UFFICIO' : 'IDENTIFICATIVO_UFFICIO',
'AIUTO|SOGGETTO_CONCEDENTE' : 'CONCEDENTE',
'AIUTO|COR' : 'COR',
'AIUTO|TITOLO_PROGETTO' : 'TITOLO_PROGETTO',
'AIUTO|DESCRIZIONE_PROGETTO' : 'DESCRIZIONE_PROGETTO',
'AIUTO|LINK_TRASPARENZA_NAZIONALE' : 'LINK_TRASPARENZA_NAZIONALE',
'AIUTO|DATA_CONCESSIONE' : 'CONCESSIONE',
'AIUTO|CUP' : 'CUP',
'AIUTO|DENOMINAZIONE_BENEFICIARIO' : 'IMPRESA',
'AIUTO|CODICE_FISCALE_BENEFICIARIO' : 'CF',
'AIUTO|DES_TIPO_BENEFICIARIO' : 'DES_TIPO_BENEFICIARIO',
'AIUTO|REGIONE_BENEFICIARIO' : 'REGIONE',
'AIUTO|IDENTIFICATIVO_MISURA_ATTUATIVA' : 'IDENTIFICATIVO_MISURA_ATTUATIVA',
'AIUTO|MISURA_ATTUATIVA_ENTE_COMPETENTE' : 'MISURA_ATTUATIVA_ENTE_COMPETENTE',
'AIUTO|MISURA_ATTUATIVA_TITOLO' : 'MISURA_ATTUATIVA_TITOLO',
'AIUTO|MISURA_ATTUATIVA_BASE_GIURIDICA_NAZIONALE' : 'MISURA_ATTUATIVA_BASE_GIURIDICA_NAZIONALE',
'AIUTO|LINK_TESTO_INTEGRALE_MISURA' : 'LINK_TESTO_INTEGRALE_MISURA',
'AIUTO|ATTO_CONCESSIONE' : 'ATTO_CONCESSIONE',
'AIUTO|LINK_ATTO_CONCESSIONE' : 'LINK_ATTO_CONCESSIONE',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|ID_COMPONENTE_AIUTO' : 'ID_COMPONENTE_AIUTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|COD_PROCEDIMENTO' : 'COD_PROCEDIMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|DES_PROCEDIMENTO' : 'DES_PROCEDIMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|COD_REGOLAMENTO' : 'COD_REGOLAMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|DES_REGOLAMENTO' : 'DES_REGOLAMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|COD_OBIETTIVO' : 'COD_OBIETTIVO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|DES_OBIETTIVO' : 'DESSETTORE_ATTIVITA_OBIETTIVO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|SETTORE_ATTIVITA' : 'SETTORE_ATTIVITA',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|SOGGETTO_INTERMEDIARIO' : 'SOGGETTO_INTERMEDIARIO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|ENTITA_DELEGATA' : 'ENTITA_DELEGATA',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|STRUMENTI_AIUTO|STRUMENTO_AIUTO|COD_STRUMENTO' : 'COD_STRUMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|STRUMENTI_AIUTO|STRUMENTO_AIUTO|DES_STRUMENTO' : 'DES_STRUMENTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|STRUMENTI_AIUTO|STRUMENTO_AIUTO|ELEMENTO_DI_AIUTO' : 'AIUTO',
'AIUTO|COMPONENTI_AIUTO|COMPONENTE_AIUTO|STRUMENTI_AIUTO|STRUMENTO_AIUTO|IMPORTO_NOMINALE': 'NOMINALE'
}, inplace = True)

df['CONCESSIONE'] = df['CONCESSIONE'].str[:-6]
df['CONCESSIONE'] = pd.to_datetime(df['CONCESSIONE'])
df['CONCEDENTE'] = df['CONCEDENTE'].str.upper() 
df['AIUTO'] = df['AIUTO'].astype(float)
df['NOMINALE'] = df['NOMINALE'].astype(float)

#df.to_parquet(periodo+'.parquet')
#df = pl.read_parquet(periodo+'.parquet')

df = df[['CAR', 'MISURA', 'COD_CE_MISURA',
       'BASE_GIURIDICA_NAZIONALE', 'CONCEDENTE',
       'COR', 'TITOLO_PROGETTO', 'DESCRIZIONE_PROGETTO',
       'CONCESSIONE', 'ATTO_CONCESSIONE',
       'IMPRESA', 'CF', 'DES_TIPO_BENEFICIARIO',
       'REGIONE', 'ID_COMPONENTE_AIUTO', 'DES_PROCEDIMENTO',
       'DESSETTORE_ATTIVITA_OBIETTIVO', 'SETTORE_ATTIVITA',
       'DES_STRUMENTO', 'AIUTO', 'NOMINALE']]
df.to_parquet('OpenData_Aiuti_'+periodo+'.parquet')

winsound.Beep(440, 300)

print('OpenData_Aiuti_'+periodo+' fatto')

OpenData_Aiuti_2018_12 fatto
CPU times: total: 42 s
Wall time: 48.3 s


## backup

In [39]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import duckdb
import os
os.chdir('D:/duckdb/files/RNA2/')
 
con = duckdb.connect()
con.execute("PRAGMA threads=2") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata


df = con.execute("select * FROM 'OpenData_Aiuti_2018*.parquet' ;").df()
df.to_parquet('anno_2018.parquet')